In [5]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

In [6]:
# Load the catalog with earthquake start times
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
catalog = pd.read_csv(cat_file)

# Directory containing the CSV files for each day
data_directory = './data/lunar/training/data/S12_GradeA/'

In [15]:
# List to store all features and targets from all days
all_features = []
all_targets = []

In [16]:
# Process each CSV file (representing one day) using the catalog to get the earthquake start time
for i, row in catalog.iterrows():
    csv_filename = row['filename'] + '.csv'
    earthquake_start_time = row['time_rel(sec)']
    
    # Load the corresponding seismic data for the day
    csv_file_path = os.path.join(data_directory, csv_filename)
    if os.path.exists(csv_file_path):
        day_data = pd.read_csv(csv_file_path)

        # Convert time to datetime and normalize time_rel
        day_data['time_abs'] = pd.to_datetime(day_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])
        day_data['time_rel_normalized'] = day_data['time_rel(sec)'] / (24 * 3600)  # Normalize to 24 hours (in seconds)

        # Apply noise reduction using Savitzky-Golay filter
        day_data['velocity_smoothed'] = savgol_filter(day_data['velocity(m/s)'], window_length=51, polyorder=3)

        # Feature engineering - windowed mean and standard deviation
        window_size = 100
        day_data['velocity_mean'] = day_data['velocity_smoothed'].rolling(window=window_size).mean()
        day_data['velocity_std'] = day_data['velocity_smoothed'].rolling(window=window_size).std()
        day_data.dropna(inplace=True)  # Drop NaN values after rolling window

        # Ensure we have enough rows for features
        if not day_data.empty:
            # Collect the features for this day
            features = day_data[['velocity_mean', 'velocity_std']].values
            all_features.append(features)
            
            # Create a target array with the same length as the number of rows in the day's data
            targets = np.full(shape=(features.shape[0],), fill_value=earthquake_start_time)
            all_targets.append(targets)

In [17]:
# Combine all data into a single dataset
all_features = np.vstack(all_features)
all_targets = np.array(all_targets)

# Split the combined data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_targets, test_size=0.3, random_state=42)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (75,) + inhomogeneous part.

In [ ]:






# Combine all data into a single dataset
all_features = np.vstack(all_features)
all_targets = np.hstack(all_targets)

# Split the combined data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_targets, test_size=0.3, random_state=42)

# Train a Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = regressor.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mean Absolute Error (MAE): {mae:.2f} seconds")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f} seconds")

# Predict the seismic event start time for a new day
new_day_filename = 'example_day.csv'  # Example of a new file
new_day_data = pd.read_csv(os.path.join(data_directory, new_day_filename))

# Apply the same preprocessing to the new data
new_day_data['time_abs'] = pd.to_datetime(new_day_data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'])
new_day_data['time_rel_normalized'] = new_day_data['time_rel(sec)'] / new_day_data['time_rel(sec)'].max()
new_day_data['velocity_smoothed'] = savgol_filter(new_day_data['velocity(m/s)'], window_length=51, polyorder=3)

new_day_data['velocity_mean'] = new_day_data['velocity_smoothed'].rolling(window=window_size).mean()
new_day_data['velocity_std'] = new_day_data['velocity_smoothed'].rolling(window=window_size).std()
new_day_data.dropna(inplace=True)

# Extract features for the new day
new_features = new_day_data[['velocity_mean', 'velocity_std']].values

# Predict the start time of the earthquake for the new day
predicted_start_time = regressor.predict(new_features)

# Plot the predicted seismic start time on the smoothed data
plt.figure(figsize=(10, 6))
plt.plot(new_day_data['time_rel_normalized'], new_day_data['velocity_smoothed'], label='Velocity Smoothed')
plt.axvline(x=predicted_start_time[0] / new_day_data['time_rel(sec)'].max(), color='red', linestyle='--', label='Predicted Start Time')
plt.title("Seismic Event Prediction for New Day")
plt.xlabel("Normalized Time")
plt.ylabel("Velocity (m/s)")
plt.legend()
plt.show()
